In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd # 2.x to support pyarrow
import pyarrow as pa

In [5]:
messages = pd.read_excel('../data/source/messages.xlsx', index_col=0)

In [6]:
messages

,МАШИНА,ИМЯ_МАШИНЫ,ТЕХ_МЕСТО,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ
390,AA2/006-006,ЭКСГАУСТЕР А/М №9,AA2/006-006-002-008,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,M3,неисправен двигатель,2019-01-21 00:00:00,2019-02-25 00:00:00,NaN
391,CH-AGP-AG2/011-005,ЭКСГАУСТЕР А/М №9,CH-AGP-AG2/011-005-002,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,M3,неисправен двигатель,2019-01-21 12:26:08,2019-02-25 00:00:00,NaN
392,AA2/002-006,ЭКСГАУСТЕР А/М №5,AA2/002-006-002-004,МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №5,M3,менять прокладку на подачу воды в охлади,2019-01-21 15:35:26,2019-01-23 11:00:00,NaN
393,CH-AGP-AG2/003-005,ЭКСГАУСТЕР А/М №5,CH-AGP-AG2/003-005-002-004,МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №5,M3,менять прокладку на подачу воды в охлади,2019-01-21 16:33:00,2019-01-22 00:00:00,NaN
394,CH-AGP-AG2/003-005,ЭКСГАУСТЕР А/М №5,CH-AGP-AG2/003-005-002-004,МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №5,M3,менять прокладку на подачу воды в охлади,2019-01-21 16:36:01,2019-06-04 15:35:26,NaN
...,...,...,...,...,...,...,...,...,...
2155,AA2/005-006,ЭКСГАУСТЕР А/М №8,AA2/005-006-001-005-001,ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №8,M3,Концевой выключатель на открытие г.задв.,2021-12-28 09:20:00,2021-12-30 00:00:00,NaN
2156,AA2/005-006,ЭКСГАУСТЕР А/М №8,AA2/005-006-003-002,РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №8,M3,Смазка редуктора.,2021-12-28 09:23:00,NaT,NaN
2157,AA2/003-006,ЭКСГАУСТЕР А/М №6,AA2/003-006-001-001,КОРПУС ЭКСГ. №6,M3,Прососы,2021-12-28 12:23:00,NaT,NaN
2158,AA2/001-006,ЭКСГАУСТЕР А/М №4,AA2/001-006-001-006,УЛИТА ЭКСГ. №4,M3,Прососы,2021-12-28 12:39:00,NaT,NaN


In [ ]:
x_structure = pd.read_parquet('../data/pipeline/x_structure.parquet')
y_structure = pd.read_parquet('../data/pipeline/y_structure.parquet')

In [ ]:
x_train_pretty_1h = pd.read_parquet('../data/pipeline/x_train_pretty_1h.parquet')
x_test_pretty_1h = pd.read_parquet('../data/pipeline/x_test_pretty_1h.parquet')
y_train_tte_1h = pd.read_parquet('../data/pipeline/y_train_tte_1h.parquet')
p_test_tte_1h = pd.read_parquet('../data/pipeline/p_test_tte_1h.parquet')

In [ ]:
MAX_TTE = 31 * 24 * 60 * 60

In [ ]:
sensors_values = pd.concat([x_train_pretty_1h, x_test_pretty_1h])
sensors_values = sensors_values.set_index(['DT', 'ИМЯ МАШИНЫ']).sort_index().reset_index()
sensors_values['machine_id'] = sensors_values['ИМЯ МАШИНЫ'].str.extract('№(\d+)').astype(int)
sensors_values = sensors_values.drop(['ИМЯ МАШИНЫ', 'ROW_IDX', 'DATASET_IDX'], axis=1)
sensors_values = sensors_values.sort_index(axis=1)
sensors_values.to_csv('../data/pipeline/db/sensor_values.csv', index=False)

In [ ]:
tmp = y_train_tte_1h
tmp = tmp.set_index(['DT', 'ИМЯ МАШИНЫ']).sort_index().reset_index()
tmp['machine_id'] = tmp['ИМЯ МАШИНЫ'].str.extract('№(\d+)').astype(int)
tmp = tmp.drop(['ИМЯ МАШИНЫ', 'ROW_IDX_x', 'DATASET_IDX_x', 'ROW_IDX_y', 'DATASET_IDX_y'], axis=1)
tmp = tmp.sort_index(axis=1)

tm_m1_columns = y_structure.index.map(lambda x: x + ' TTE M1')
tm_m1_values = tmp[['DT', 'machine_id']].copy()
tm_m1_values[y_structure.index] = tmp[tm_m1_columns]
tm_m1_values.to_csv('../data/pipeline/db/tm_m1_values.csv', index=False)

tm_m3_columns = y_structure.index.map(lambda x: x + ' TTE M3')
tm_m3_values = tmp[['DT', 'machine_id']].copy()
tm_m3_values[y_structure.index] = tmp[tm_m3_columns]
tm_m3_values.to_csv('../data/pipeline/db/tm_m3_values.csv', index=False)

In [ ]:
# @WIP: m1/m3 predictions